## data samples AGG -
|train | test | dev |
|----|---| ---|
|56355| 8421 | 15878 |
|16355| 2505 | 4722 |

In [1]:
from utils.consts import *

from utils.utilities import (
    test, show_dep, get_data, get_nominal_subjects, get_prep, get_objects, get_root, _is_verb, get_aux,
    check_if_wh, get_csubj, get_amod, get_actual_subtree_for_finding_agg)

import pickle
import json
import random
import json
from IPython.display import Markdown, display
from pprint import pprint
import numpy as np

In [5]:
import spacy

# nlp_pipe = spacy.load('en_core_web_lg')
PIPE = spacy.load('en_coref_lg')

In [6]:
def get_verb_and_subj_pair(data_sample):
#     print("\n========================================================\n")
#     pprint(data_sample)
    # get the doc representation of the string
    doc = PIPE(data_sample['question'])
    # Find out the root of the given statement
    root = get_root(doc)
#     import pdb; pdb.set_trace()
    is_verb = _is_verb(root)
    aux = None
#     if not is_verb:
        # Some times the root can be a noun, in that case it'll always be associated to
        # an aux verb, and we gotta find it
    aux = get_aux(root)
    # If there is any aux found, we use it as our verb root
    if aux is not None and len(aux) > 0:
        root_verb = aux[0]
    else:
        # Else, we use our root as it is
        root_verb = root
    # Check if there is any csubj attached this means we have two wh nouns attached to
    # two different verbs one is asking and other qualifying.
    csubj_verb = get_csubj(root_verb)
    if csubj_verb is not None:
        root_verb = csubj_verb
#     print("ROOT : {}".format(root.text))
#     print("AUX : ")
#     pprint(aux)
    # nominals in this case is going to be a list of all the cols
    # in SELECT clause. If we find any nsubj attached to the root_verb we catch it here.
    nominals = get_nominal_subjects(root_verb)
    is_wh = False
    # If what we have found is not actually a noun but a question noun (wh-word)
    # TODO: In this case we may want to go on hunt for the actual nominal col.
    if len(nominals) != 0:
        is_wh = check_if_wh(nominals)
    if is_wh:
        # for now
        pass
#     print("SUBJ : ")
#     pprint(nominals)
    # Even after all this if there is no way the prog have found a nominal
    # find out if there is any prop attached to the verb.
    # If there is then we may look for prep_obj attached to it for the nominal.
    prep = None
    if len(nominals) == 0:
        prep = get_prep(root_verb)
#     print("PREP : ")
#     pprint(prep)
    pobj = None
    if prep is not None:
        pobj = get_objects(prep)
        nominals.append(pobj)
#     print("POBJ : ")
#     pprint(pobj)
#     show_dep(doc)
    return nominals

In [7]:
def dump_data():
    agg_X = np.array(agg_X)
    agg_Y = np.array(agg_Y)

    with open(x_file, 'wb') as fx:
        pickle.dump(agg_X, fx)

    with open(y_file, 'wb') as fy:
        pickle.dump(agg_Y, fy)

In [8]:
dataset = './data/test_dataset.json'
x_file = './test_agg_X.pkl'
y_file = './test_agg_Y.pkl'

data = get_data(dataset)

In [9]:
X_size = 1500
agg_X = list()
agg_Y = list()
agg_funcs = np.array(agg_ops)

def fill_data(act_st, act_agg, idx):
    temp_x = np.array([])
    temp_y = np.array([])
    if act_st is not None and len(act_st) > 0 and act_agg in agg_ops:
#         print("idx : %d"%idx)
        act_st = act_st[:5]
        for tok in act_st:
            if tok.has_vector:
                temp_x = np.append(temp_x, tok.vector)
        temp_y = (agg_funcs == act_agg).astype(np.float)
    else:
        temp_x = np.zeros((X_size))
        temp_y = (agg_funcs == "").astype(np.float)
    if temp_x.size < X_size:
        temp_x = np.append(temp_x, np.zeros((X_size - temp_x.size)))
    
    agg_X.append(temp_x)
    agg_Y.append(temp_y)

In [11]:
count = 0
for idx in range(len(data[:10])):
    conds = data[idx]['sql']['conds']
    try:
        subjs = get_verb_and_subj_pair(data[idx])
        if agg_func.strip() == "":
            continue
        print("\n========================================================\n")
        pprint(data[idx])
        show_dep(PIPE(data[idx]['question']))
        print("SUBJs : ")
        pprint(subjs)
        print("SELECT : ")
        pprint(data[idx]['sql']['sel'])
        print("CONDS : ", conds)
        count += 1
#         fill_data(act, agg_func, idx)
    except Exception:
#         amod = get_amod(PIPE(data[idx]['question']))
#         fill_data(amod, agg_func, idx)
        pass

print(count)
#         pprint(data[idx])




{'header': ['Player',
            'No.',
            'Nationality',
            'Position',
            'Years in Toronto',
            'School/Club Team'],
 'phase': 1,
 'question': "What is terrence ross' nationality",
 'sql': {'agg': '',
         'conds': [['Player', '=', 'Terrence Ross']],
         'sel': 'Nationality'},
 'table_id': '1-10015132-16'}


SUBJs : 
[]
SELECT : 
'Nationality'
CONDS :  [['Player', '=', 'Terrence Ross']]


{'header': ['Player',
            'No.',
            'Nationality',
            'Position',
            'Years in Toronto',
            'School/Club Team'],
 'phase': 1,
 'question': 'What clu was in toronto 1995-96',
 'sql': {'agg': '',
         'conds': [['Years in Toronto', '=', '1995-96']],
         'sel': 'School/Club Team'},
 'table_id': '1-10015132-16'}


SUBJs : 
[clu]
SELECT : 
'School/Club Team'
CONDS :  [['Years in Toronto', '=', '1995-96']]


{'header': ['Player',
            'No.',
            'Nationality',
            'Position',
            'Years in Toronto',
            'School/Club Team'],
 'phase': 1,
 'question': 'which club was in toronto 2003-06',
 'sql': {'agg': '',
         'conds': [['Years in Toronto', '=', '2003-06']],
         'sel': 'School/Club Team'},
 'table_id': '1-10015132-16'}


SUBJs : 
[club]
SELECT : 
'School/Club Team'
CONDS :  [['Years in Toronto', '=', '2003-06']]


{'header': ['Player',
            'No.',
            'Nationality',
            'Position',
            'Years in Toronto',
            'School/Club Team'],
 'phase': 1,
 'question': 'how many schools or teams had jalen rose',
 'sql': {'agg': 'COUNT',
         'conds': [['Player', '=', 'Jalen Rose']],
         'sel': 'School/Club Team'},
 'table_id': '1-10015132-16'}


SUBJs : 
[]
SELECT : 
'School/Club Team'
CONDS :  [['Player', '=', 'Jalen Rose']]


{'header': ['No',
            'Date',
            'Round',
            'Circuit',
            'Pole Position',
            'Fastest Lap',
            'Race winner',
            'Report'],
 'phase': 1,
 'question': 'Where was Assen held?',
 'sql': {'agg': '', 'conds': [['Circuit', '=', 'Assen']], 'sel': 'Round'},
 'table_id': '1-10083598-1'}


...

SUBJs : 
[number]
SELECT : 
'No'
CONDS :  [['Pole Position', '=', 'Kevin Curtain']]


{'header': ['No',
            'Date',
            'Round',
            'Circuit',
            'Pole Position',
            'Fastest Lap',
            'Race winner',
            'Report'],
 'phase': 1,
 'question': 'What was the date of the race in Misano?',
 'sql': {'agg': '', 'conds': [['Circuit', '=', 'Misano']], 'sel': 'Date'},
 'table_id': '1-10083598-1'}


SUBJs : 
[date]
SELECT : 
'Date'
CONDS :  [['Circuit', '=', 'Misano']]


{'header': ['Pick',
            'Player',
            'Position',
            'Nationality',
            'NHL team',
            'College/junior/club team'],
 'phase': 1,
 'question': 'How many different positions did Sherbrooke Faucons (qmjhl) '
             'provide in the draft?',
 'sql': {'agg': 'COUNT',
         'conds': [['College/junior/club team',
                    '=',
                    'Sherbrooke Faucons (QMJHL)']],
         'sel': 'Position'},
 'table_id': '1-1013129-2'}


SUBJs : 
[positions]
SELECT : 
'Position'
CONDS :  [['College/junior/club team', '=', 'Sherbrooke Faucons (QMJHL)']]


{'header': ['Pick',
            'Player',
            'Position',
            'Nationality',
            'NHL team',
            'College/junior/club team'],
 'phase': 1,
 'question': 'What are the nationalities of the player picked from Thunder Bay '
             'Flyers (ushl)',
 'sql': {'agg': '',
         'conds': [['College/junior/club team',
                    '=',
                    'Thunder Bay Flyers (USHL)']],
         'sel': 'Nationality'},
 'table_id': '1-1013129-2'}


SUBJs : 
[nationalities]
SELECT : 
'Nationality'
CONDS :  [['College/junior/club team', '=', 'Thunder Bay Flyers (USHL)']]


{'header': ['Pick',
            'Player',
            'Position',
            'Nationality',
            'NHL team',
            'College/junior/club team'],
 'phase': 1,
 'question': 'How many different college/junior/club teams provided a player '
             'to the Washington Capitals NHL Team?',
 'sql': {'agg': 'COUNT',
         'conds': [['NHL team', '=', 'Washington Capitals']],
         'sel': 'College/junior/club team'},
 'table_id': '1-1013129-2'}


SUBJs : 
[teams]
SELECT : 
'College/junior/club team'
CONDS :  [['NHL team', '=', 'Washington Capitals']]
10


In [24]:
agg_X[:10]

array([[ 0.091336  , -0.026953  , -0.38925001, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02483   ,  0.32008001, -0.24504   , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.091336  , -0.026953  , -0.38925001, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.57609999,  0.43094999,  0.69146001, ...,  0.        ,
         0.        ,  0.        ],
       [-0.57609999,  0.43094999,  0.69146001, ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
show_dep(PIPE("What is the English name of the country whose official native language is Dutch Papiamento?"))

Find csubj for the verb
[displacy link](https://explosion.ai/demos/displacy?text=What%20is%20the%20English%20name%20of%20the%20country%20whose%20official%20native%20language%20is%20Dutch%20Papiamento%3F&model=en_core_web_lg&cpu=1&cph=0)
[coref](https://huggingface.co/coref/?text=What%20is%20the%20English%20name%20of%20the%20country%20whose%20official%20native%20language%20is%20Dutch%20Papiamento%3F)
[dep list](https://spacy.io/api/annotation#section-dependency-parsing)
